# Analyse av tekster for Polnet
Lars G. Johnsen
Yngvil Beyer
Kirsten Sivesind


I den her notebooken setter vi opp en kobling til tekstene som ligger i Zotero-biblioteket for Polnet-prosjektet. I første omgang prøver vi å lage kollokasjoner mot tekstene gjennom API mot NB-digital, men på sikt kan vi lage en database over alle PDF-er som ligger i Zotero https://www.zotero.org/groups/2438311/polnet2/library.





Aktiver moduler for å hente tekst og analysere

In [28]:
import dhlab.nbtext as nb
from dhlab.nbtokenizer import tokenize
import json
import pandas as pd
from collections import Counter
import re

In [2]:
from dhlab.module_update import css
css()

In [71]:
def collocations_from_nb(word, corpus):
    """Get a concordance, and count the words in it. Assume konks reside a dataframe with columns 'after' and 'before'"""
    concordance = nb.frame(nap.get_all_konks(word, korpus))
    return nb.frame_sort(nb.frame(Counter(tokenize(' '.join(concordance['after'].values + concordance['before'].values))), word))

def count_from_conc(concordance):
    """From a concordance, count the words in it. Assume konks reside a dataframe with columns 'after' and 'before'"""
    word = concordance['word'][0]
    return nb.frame_sort(nb.frame(Counter(tokenize(' '.join(concordance['after'].values + concordance['before'].values))), word))

Modulen `nb_api` gir adgang til tekstene på samme måte som i bokhylla, eller https://nb.no

In [3]:
import nb_api as nap

Metadata fra Zotero brukes om guide til å hente tekster og lage subkorpus. Filen bør ligge i samme mappe som notebooken.

In [8]:
zotero = pd.read_excel("Zoterobibliotek eksportert_010420.xlsx").fillna('')
zotero.head()

,Key,Source ID,Column1,Item Type,Publication Year,Author,Title,Publication Title,ISBN,ISSN,...,Programming Language,Version,System,Code,Code Number,Section,Session,Committee,History,Legislative Body
0,I8XV7B38,,,document,1957,"Kirke,-undervisningsdepartementet",St.meld. nr. 35 (1957) Om lærebøkene i skoleve...,,,,...,,,,,,,,,,
1,NA5JTD78,,,bookSection,1970,"Kirke,-undervisningsdepartementet",St. meld. nr. 91 (1969—70) Om reform av gymnas...,,,,...,,,,,,,,,,
2,235TMJP4,,,document,1972,Forbruker- og administrasjonsdepartementet,NOU 1972: 39 Førskoler : en utredning,,,,...,,,,,,,,,,
3,TK8NAQ9C,,,book,1974,"Kirke- og undervisningsdepartementet; Østerud,...",NOU: 1974: 58 Lærarutdanning: tilråding I og II,,978-82-00-70182-8,,...,,,,,,,,,,
4,EXM5QBAL,,,document,1974,Kirke- og undervisningsdepartementet,St.meld. nr. 17 (1973—74) Nordisk samarbeid.,,,,...,,,,,,,,,,


Kolonnene er de samme som ligger i Zotero.

In [17]:
zotero.columns

Index(['Key', 'Source ID', 'Column1', 'Item Type', 'Publication Year',
       'Author', 'Title', 'Publication Title', 'ISBN', 'ISSN', 'DOI', 'Url',
       'Abstract Note', 'Date', 'Date Added', 'Date Modified', 'Access Date',
       'Pages', 'Num Pages', 'Issue', 'Volume', 'Number Of Volumes',
       'Journal Abbreviation', 'Short Title', 'Series', 'Series Number',
       'Series Text', 'Series Title', 'Publisher', 'Place', 'Language',
       'Rights', 'Type', 'Archive', 'Archive Location', 'Library Catalog',
       'Call Number', 'Extra', 'Notes', 'File Attachments', 'Link Attachments',
       'Manual Tags', 'Automatic Tags', 'Editor', 'Series Editor',
       'Translator', 'Contributor', 'Attorney Agent', 'Book Author',
       'Cast Member', 'Commenter', 'Composer', 'Cosponsor', 'Counsel',
       'Interviewer', 'Producer', 'Recipient', 'Reviewed Author',
       'Scriptwriter', 'Words By', 'Guest', 'Number', 'Edition',
       'Running Time', 'Scale', 'Medium', 'Artwork Size', 'Filing D

Dataene er utstyrt med noen tagger. Tell opp og se.

In [23]:
tags = nb.frame_sort(nb.frame(Counter(zotero['Manual Tags'].values), 'antall'.split()))
tags


,antall
,249
Høyere utdanning,24
Grunnskole,17
Lærerutdanning,9
Videregående opplæring; Grunnskole,9
Videregående opplæring,5
Barnehage,4
Voksenopplæring,2
St.meld.nr.39. Om kunst og kultur i og i tilknytning til grunnskolen. 2003.,2
Videregående opplæring; Barnehage; Grunnskole,2


### Analyse av data

Henter ut referanser fra zoterodataene for å komme til NB-digital. 

Listen over NB-data består av identifikatorer på sesamidformat eller URN-format. 

**Viktig: NB-digital har samleverk for statsmaktene, så søk i på sesamid tar hele samlingen, mens det kun er ett dokument som egentlig tilhører Polnet.** 

In [27]:
dokref = zotero['Url'][zotero['Url'].str.contains('nb.no/')].values
dokref

array(['https://www.nb.no/statsmaktene/nb/5c15bbeeba31f51f2eeaf760d831cb99?index=11#1161',
       'https://www.nb.no/items/URN:NBN:no-nb_digibok_2008052700008',
       'https://urn.nb.no/URN:NBN:no-nb_digibok_2013100805016',
       'https://www.nb.no/statsmaktene/nb/ca49bb43eeb537f04d71bb76bf978663?index=3#1231',
       'https://urn.nb.no/URN:NBN:no-nb_digibok_2012062606119',
       'https://urn.nb.no/URN:NBN:no-nb_digibok_2012062606119',
       'https://www.nb.no/statsmaktene/nb/22ade39f6909f817f358a81a16ca1337?index=144#635',
       'https://www.nb.no/statsmaktene/nb/45615976d5e5bff65500f7f8bb7fcf87?index=2#1285',
       'https://urn.nb.no/URN:NBN:no-nb_digibok_2013100106023',
       'https://www.nb.no/statsmaktene/nb/cc4e6f1a51d0b2b7b0f7ee91b2725dc0?index=1#717',
       'https://urn.nb.no/URN:NBN:no-nb_digibok_2012071106024',
       'https://urn.nb.no/URN:NBN:no-nb_digibok_2013100106024',
       'https://urn.nb.no/URN:NBN:no-nb_digibok_2013120205188',
       'https://urn.nb.no/URN:N

Henter først ut en liste over sesamid, ser på de fem første.

In [43]:
sesamids = [x.split('nb/')[-1].split('?')[0] for x in dokref if 'statsmaktene' in x]
sesamids[:5]

['5c15bbeeba31f51f2eeaf760d831cb99',
 'ca49bb43eeb537f04d71bb76bf978663',
 '22ade39f6909f817f358a81a16ca1337',
 '45615976d5e5bff65500f7f8bb7fcf87',
 'cc4e6f1a51d0b2b7b0f7ee91b2725dc0']

så en liste over URN-er

In [46]:
urns = [x.split('/')[-1] for x in dokref if 'URN:NBN' in x ]
urns[:5]

['URN:NBN:no-nb_digibok_2008052700008',
 'URN:NBN:no-nb_digibok_2013100805016',
 'URN:NBN:no-nb_digibok_2012062606119',
 'URN:NBN:no-nb_digibok_2012062606119',
 'URN:NBN:no-nb_digibok_2013100106023']

Legger sammen listene, og holder dem i variabelen `dok_ids`

In [47]:
dok_ids = sesamids + urns

Med identifikatorene på plass, kan det hentes ut konkordanser og lages kollokasjoner direkte.

Konkordanser hentes rett fra bokhylla for ett og ett verk, eller grupperes og summeres.

#### Enkeltverk

In [48]:
kompetanse_konk = nb.frame(nap.get_konks('a7923729d620364642d44e105d7ff956','kompetanse'))
kompetanse_konk

,urn,before,word,after
0,URN:NBN:no-nb_digistorting_1990-91_part3_vol-c...,utgjøre grunnlaget for samarbeidsavtaler som a...,kompetanse,disse organisasj onene har i makroøkonomiske ...
1,URN:NBN:no-nb_digistorting_1990-91_part3_vol-c...,å sette igang et omfattende nordisk energifors...,kompetanse,innenfor energiområdet ved de nasjonale høysk...
2,URN:NBN:no-nb_digistorting_1990-91_part3_vol-c...,en økende ulykkesfrekvens i prosesskjemisk ind...,kompetanse,og sikkerhetsbevissthet i bedriftene. Man har...
3,URN:NBN:no-nb_digistorting_1990-91_part3_vol-c...,. 38 7.6 Bærekraftig ressursforvaltning 40 7.7...,Kompetanse,- nøkkelen til økt verdiskapning og lønnsomhe...
4,URN:NBN:no-nb_digistorting_1990-91_part3_vol-c...,vekst i sysselsettingen .. 78 10.3 Internasjon...,Kompetanse,er viktig for utvikling av næringslivet 80 10...
...,...,...,...,...
237,URN:NBN:no-nb_digistorting_1990-91_part3_vol-c...,siktemål må være at et lærerkollegium skal sty...,kompetanse.,Skolens behov må da komme til uttrykk ved utl...
238,URN:NBN:no-nb_digistorting_1990-91_part3_vol-c...,"deltidsstudium og i moduler, slik at lærerne k...",kompetanse,over tid. Departementet vil legge til rette f...
239,URN:NBN:no-nb_digistorting_1990-91_part3_vol-c...,eller arbeidsområder som er relevante for lære...,kompetanse,gjennom forskning og utviklingsarbeid. Med de...
240,URN:NBN:no-nb_digistorting_1990-91_part3_vol-c...,fagmiljøer som i dag er spredt på flere høgsko...,kompetanse,og økt satsing på forsknings- og utviklingsar...


#### For en samling

In [49]:
konks_reform = nb.frame(nap.get_all_konks('reform', dok_ids))

In [50]:
konks_fornyelse = nb.frame(nap.get_all_konks('fornyelse', dok_ids))

In [51]:
konks_reform

,urn,before,word,after
0,URN:NBN:no-nb_digistorting_1969-70_part3_vol-c...,Lønns- og inntektsoppgjørene våren 1970. 91. Om,reform,av gymnaset og forsøk med utbygging av det vi...
1,URN:NBN:no-nb_digistorting_1969-70_part3_vol-c...,og inntektsoppgjørene våren 1970. Kirke- og un...,reform,av gymnaset og forsøk med utbygging av det vi...
2,URN:NBN:no-nb_digistorting_1969-70_part3_vol-c...,og Skolekomitéens innstillinger 34 Kap. 111. E...,reform,37 Vedlegg. 1. Uttaleleer om innstilling I og...
3,URN:NBN:no-nb_digistorting_1969-70_part3_vol-c...,elevtall i det videregående skoleverk 67 3. Ut...,reform,av gymnaset frå utvalet til å vurdere
4,URN:NBN:no-nb_digistorting_1969-70_part3_vol-c...,i lov om realskoler og gymnas. 108,Reform,og forsak i yrkesskolen 115 6. Om etterutdann...
...,...,...,...,...
2310,URN:NBN:no-nb_digibok_2011040406096_0486,"Oslo På rett vei, uten rett.",Reform,94 - konsekvenser for voksne. Sluttrapport. N...
2311,URN:NBN:no-nb_digibok_2011040406096_0487,undersøkelse av den fylkeskommunale oppfølging...,Reform,94. Sammendrag fra sluttrapportene fra KUF Ve...
2312,URN:NBN:no-nb_digibok_2011091906185_0055,og den vidaregåande opplæring. 64 For voksne s...,Reform,"94), gir § 4A-1 rett til grunnskoleopplæ- rin..."
2313,URN:NBN:no-nb_digibok_2011091906185_0085,"spesifikk bestemmelse, må statene rapportere s...",reform,i 1998 håndheves EMK av Den europeiske mennes...


# Telling for kollokasjonsanalyse

For tellingen legger vi sammen alle konkordansene til en eneste stor tekst, som tokeniseres og telles opp på vanlig måte.


In [57]:
konks_reform['after'].values + konks_reform['before'].values

array([' av gymnaset og forsøk med utbygging av det videregående skoleverk. Vedlegg: 1. Tilråding om Lønns- og inntektsoppgjørene våren 1970. 91. Om ',
       ' av gymnaset og forsøk med utbygging av det videregående skoleverk. INNHOLD Side Kap. I.og inntektsoppgjørene våren 1970. Kirke- og undervisningsdepartementet. St. meld. nr. 91. (1969—70) Om ',
       ' 37 Vedlegg. 1. Uttaleleer om innstilling I og II fra Skolekomitéen av 1965 40 Struktureringog Skolekomitéens innstillinger 34 Kap. 111. En del forutsetninger for gjennomføring av forsøk og ',
       ...,
       ' 94), gir § 4A-1 rett til grunnskoleopplæ- ring ved behov. Voksne kan også få rett til videregåendeog den vidaregåande opplæring. 64 For voksne som ikke omfattes av retten til videregående opplæring (',
       ' i 1998 håndheves EMK av Den europeiske menneskerettsdomstol (EMD) . 40 Både stater og individer kanspesifikk bestemmelse, må statene rapportere særskilt om denne gjennomføringen til barnekomiteen. 38 Etter en ',
 

Så til opptellingen

In [75]:
coll_reform = nb.frame_sort( nb.frame(Counter(tokenize(' '.join(konks_reform['after'].values + konks_reform['before'].values))), 'reform'))

In [76]:
coll_reform

,reform
.,3575
og,2127
i,2077
av,2051
",",1439
...,...
ferdig,1
utbygget,1
innebæreI,1
8.5,1


Noen hjelpefunksjoner gjør det hele litt enklere:

In [77]:
coll_fornyelse = count_from_conc(konks_fornyelse)

In [78]:
coll_fornyelse

,fornyelse
og,1998
.,1924
av,1350
i,1182
",",997
...,...
genera,1
neste,1
barnekulturen,1
spenningen,1


## Relevante ord

Sammenligner med bokhylla nå i første omgang. Det er flere måter å finne relevante ord, og flere måter å kna ordene på. Her skal vi bare bruke ordene som de er, og sammenligne med de 50000 mest frekvente i bokhylla.

In [74]:
tot = nb.frame(nb.totals(50000),'tot')

Dataene normaliseres. Siden korpusene har forskjellig størrelse, gir normaliseringen tall som kan sammenlignes.

In [79]:
nb.normalize_corpus_dataframe(tot)
nb.normalize_corpus_dataframe(coll_reform)
nb.normalize_corpus_dataframe(coll_fornyelse)

True

For eksempel er det omtrent samme antall punktum og høyfrekvente ord i dataene:

In [81]:
print(tot.loc['.'], coll_reform.loc['.'], coll_fornyelse.loc['.'])

tot    0.058921
Name: ., dtype: float64 reform    0.051051
Name: ., dtype: float64 fornyelse    0.046351
Name: ., dtype: float64


Sammenligner ved se på forholdet mellom ord i den ene vs. den andre.

In [86]:
nb.frame(coll_reform['reform']/(coll_reform['reform'] +tot['tot'])).sort_values(by=0, ascending=False).head(40)

,0
gymnaset,1225.461425
klassetrinn,459.928111
meld,409.653793
Sikkerhetsrådet,387.135616
lærlinger,368.004758
tot,358.073113
Europarådet,346.369012
skattesystemet,324.928239
FNs,284.963033
lokaler,244.120015


Alt kan puttes inn i en stor dataramme:

In [93]:
coll = pd.concat([coll_fornyelse, coll_reform], axis=1, sort=True)

Merk at når kollokasjonsteksten sorteres kommer koordinatorer høyt. 

In [98]:
nb.frame_sort(coll).head().style.background_gradient(axis=1)

,fornyelse,reform
.,0.0463514,0.051051
og,0.0481341,0.0303736
i,0.0284758,0.0296596
av,0.0325231,0.0292883
",",0.0240189,0.0205489


In [99]:
nb.frame(coll['reform']/tot['tot']).sort_values(by=0, ascending=False).head(40)

,0
gymnaset,1225.461425
klassetrinn,459.928111
meld,409.653793
Sikkerhetsrådet,387.135616
lærlinger,368.004758
tot,358.073113
Europarådet,346.369012
skattesystemet,324.928239
FNs,284.963033
lokaler,244.120015


Sammenligner de forskjellige kollokasjonene

In [109]:
pmi = nb.frame_sort(nb.frame({x:coll[x]/tot['tot'] for x in coll}).transpose())

In [114]:
pmi.head(40).fillna(0).style.background_gradient()

,fornyelse,reform
gymnaset,7.89092,1225.46
klassetrinn,0,459.928
meld,50.5689,409.654
Sikkerhetsrådet,90.0854,387.136
lærlinger,0,368.005
tot,0,358.073
Europarådet,0,346.369
skattesystemet,0,324.928
FNs,18.7792,284.963
lokaler,0,244.12
